In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import os
path = '/content/drive/MyDrive/'
os.chdir(path)

In [ ]:
import torch
import torch.nn.functional as F

outputs = torch.randn(16, 2)
print(outputs.shape)
print(outputs)

probs = F.softmax(outputs, dim=1)
print(probs)

_, selected_index = torch.max(probs, dim=1)
print(selected_index)

torch.Size([16, 2])
tensor([[-0.5530,  0.6827],
        [ 1.1760,  0.2220],
        [-0.1655,  1.2430],
        [-0.3634,  1.8711],
        [ 1.0592, -1.8296],
        [ 0.5326,  0.7684],
        [ 1.3131,  0.1916],
        [-1.8843, -0.1853],
        [ 0.5419, -0.8931],
        [ 0.0707,  0.0392],
        [ 0.5924, -0.5188],
        [ 1.1447, -0.5955],
        [-1.4452, -0.3069],
        [-0.8108, -1.1703],
        [ 0.7726,  1.2278],
        [ 0.3160, -1.3612]])
tensor([[0.2252, 0.7748],
        [0.7219, 0.2781],
        [0.1965, 0.8035],
        [0.0967, 0.9033],
        [0.9473, 0.0527],
        [0.4413, 0.5587],
        [0.7543, 0.2457],
        [0.1546, 0.8454],
        [0.8077, 0.1923],
        [0.5079, 0.4921],
        [0.7524, 0.2476],
        [0.8507, 0.1493],
        [0.2426, 0.7574],
        [0.5889, 0.4111],
        [0.3881, 0.6119],
        [0.8425, 0.1575]])
tensor([1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0])


In [13]:
import torch
import torch.optim as optim
from torchvision import transforms
from torchvision import datasets
import torch.nn as nn
import ssl
from sklearn.metrics import roc_auc_score, roc_curve, auc, confusion_matrix
from matplotlib import pyplot as plt
import math
import time
import os
import torch.nn.functional as F
from torchvision import models

ssl._create_default_https_context = ssl._create_unverified_context

In [21]:
EPOCH = 3
epochs = EPOCH
pre_epoch = 0

def plot_accuracy(acc, val_acc, loss, val_loss):
  # accuracy curve
  plt.plot(epochs, acc)
  plt.plot(epochs, val_acc)
  plt.title("Training and Validation accuracy")
  plt.legend(("Training accuracy", "Validation accuracy"))
  plt.figure()
  # loss curve
  plt.plot(epochs, loss)
  plt.plot(epochs, val_loss)
  plt.title("Training and Validation loss")
  plt.legend(("Training loss", "Validation loss"))
  plt.show()

# choose resnet101 as model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet101(pretrained=True)
model.fc = nn.Linear(in_features=2048, out_features=2, bias=True)

# data location
train_dir = "/content/drive/MyDrive/cat_and_dog/train"
val_dir = "/content/drive/MyDrive/cat_and_dog/val"

# hyperparameter setting
batch_size = 32
lr = 0.0001
model = model.to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

preprocess = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]
)

# load the data
train_data = datasets.ImageFolder(train_dir, preprocess)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

val_data = datasets.ImageFolder(val_dir, preprocess)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True)

# Define the variables


# Training process
def train():
  for epoch in range(epochs):
    start_time = time.time()
    print("\nEpoch: %d" % (epoch + 1))
    model.train()
    sum_loss = 0.0
    correct = 0.0
    total = 0.0
    train_acc = 0
    label_train = []
    prob_train = []
    length = len(train_loader)
    for index, data in enumerate(train_loader):
      images, labels = data
      images = images.to(device)
      labels = labels.to(device)
      
      preds = model(images)
      loss = loss_func(preds, labels)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      sum_loss += loss.item()
      # [16, 2] -> by rows -> dim=1
      probs = F.softmax(preds, dim=1)
      _, predicted_labels = torch.max(probs, dim=1)
      total += labels.size(0)
      correct += (predicted_labels == labels).sum()
      train_acc = correct / total
      print("[epoch: %d, iter: %d] Loss: %.03f | Acc: %.3f%% " %
            (epoch + 1, (index + 1 + epoch * length), sum_loss / (index + 1), 100. * train_acc))
      time_elapsed = time.time() - start_time
  print("Training accuracy is: %.3f%%" % (100. * train_acc))
  print("Training completed in {}s".format(time_elapsed))
  print("Waiting for test....")

def test():
  model.eval()
  test_correct = 0.0
  test_total = 0.0
  with torch.no_grad():
    for index, data in enumerate(val_loader):
      images, labels = data
      images = images.to(device)
      labels = labels.to(device)
      test_preds = model(images)
      
      # [16 , 2] -> softmax dimension one -> torch.max
      test_probs = F.softmax(test_preds, dim=1)
      _, test_predicted = torch.max(test_probs, dim=1)
      test_correct += (test_predicted == labels).sum()
      test_total += labels.size(0)
  test_acc = test_correct / test_total
  print("Test accuracy is %.3f" % test_acc)

if __name__ == "__main__":
  train()
  test()


Epoch: 1
[epoch: 1, iter: 1] Loss: 0.801 | Acc: 43.750% 
[epoch: 1, iter: 2] Loss: 0.656 | Acc: 62.500% 
[epoch: 1, iter: 3] Loss: 0.524 | Acc: 73.958% 
[epoch: 1, iter: 4] Loss: 0.434 | Acc: 80.469% 
[epoch: 1, iter: 5] Loss: 0.361 | Acc: 84.375% 
[epoch: 1, iter: 6] Loss: 0.307 | Acc: 86.979% 
[epoch: 1, iter: 7] Loss: 0.267 | Acc: 88.839% 
[epoch: 1, iter: 8] Loss: 0.240 | Acc: 90.234% 
[epoch: 1, iter: 9] Loss: 0.215 | Acc: 91.319% 
[epoch: 1, iter: 10] Loss: 0.194 | Acc: 92.188% 
[epoch: 1, iter: 11] Loss: 0.178 | Acc: 92.898% 
[epoch: 1, iter: 12] Loss: 0.163 | Acc: 93.490% 
[epoch: 1, iter: 13] Loss: 0.151 | Acc: 93.990% 
[epoch: 1, iter: 14] Loss: 0.140 | Acc: 94.420% 
[epoch: 1, iter: 15] Loss: 0.131 | Acc: 94.792% 

Epoch: 2
[epoch: 2, iter: 16] Loss: 0.007 | Acc: 100.000% 
[epoch: 2, iter: 17] Loss: 0.004 | Acc: 100.000% 
[epoch: 2, iter: 18] Loss: 0.003 | Acc: 100.000% 
[epoch: 2, iter: 19] Loss: 0.002 | Acc: 100.000% 
[epoch: 2, iter: 20] Loss: 0.002 | Acc: 100.000% 
[epo